<a href="https://colab.research.google.com/github/Jose-Gabriel-Rodriguez/MachineLearning/blob/main/Unidad3/Practica_6_SVM_Lending_Club.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://github.com/JoseGabriel-ITD/Probabilidad-y-Estadistica/blob/main/Cintilla2004.png?raw=true)

# **Machine Learning y Deep Learning**

## Unidad 3

### Practica 6: **Suport Vector Machine (SVM) Prestamos Lending Club**

### Facilitador: *Dr. José Gabriel Rodríguez Rivas*

### Alumno:  _____________


### **SVM (Máquina de Vectores de Soporte)**

* SVM (Máquina de Vectores de Soporte) es un modelo supervisado que **busca encontrar el hiperplano que mejor separa las clases** en el espacio de características.
* Ese hiperplano óptimo es aquel que maximiza el margen entre las clases, es decir, la distancia a los puntos más cercanos de cada clase (los llamados vectores de soporte).

**Tipos de SVM**
* **Lineal**: intenta separar las clases con una línea recta (2D) o un plano (3D o más dimensiones).  
* **No lineal (con kernel)**: cuando los datos no se pueden separar linealmente, el SVM transforma el espacio de entrada usando una función kernel.  

**Funciones kernel más comunes**  

| Kernel                          | Descripción                             | Parámetro principal |
| :------------------------------ | :-------------------------------------- | :------------------ |
| `'linear'`                      | Separación lineal. Rápido y simple.     | `C`                 |
| `'poly'`                        | Usa polinomios para curvas.             | `C`, `degree`       |
| `'rbf'` (radial basis function) | El más usado. Separa datos no lineales. | `C`, `gamma`        |
| `'sigmoid'`                     | Similar a redes neuronales.             | `C`, `gamma`        |

**Parámetros importantes**

| Parámetro                 | Significado                                                                                              |
| :------------------------ | :------------------------------------------------------------------------------------------------------- |
| `C`                       | Controla la penalización de errores (entre sobreajuste y generalización). Valores grandes ⇒ sobreajuste. |
| `kernel`                  | Tipo de función de transformación del espacio.                                                           |
| `gamma`                   | Controla la influencia de los puntos individuales (solo para RBF, poly y sigmoid).                       |
| `class_weight='balanced'` | Ajusta los pesos automáticamente si hay desbalance de clases.                                            |





## **Cargar librerías y dataset**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Importamos librerias necesarias para Random Forest Classifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

In [2]:
prestamos_df = pd.read_csv('prestamos_ok.csv')
prestamos_df.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,issue_year,repaid,loan_term_year,purpose_code,home_ownership_code,grade_code,addr_state_code
0,0,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,Cash,N,NaN,2011,1,3,11,4,2,2
1,1,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,Cash,N,NaN,2011,1,3,9,4,2,0
2,2,3000,3000,3000.0,36 months,18.64,109.43,E,E1,MKC Accounting,...,Cash,N,NaN,2011,1,3,0,4,4,0
3,3,5600,5600,5600.0,60 months,21.28,152.39,F,F2,NaN,...,Cash,N,NaN,2011,0,5,11,3,5,0
4,4,5375,5375,5350.0,60 months,12.69,121.45,B,B5,Starbucks,...,Cash,N,NaN,2011,0,5,9,4,1,5


## **Selección de Variables predictoras**   

**Para la seleccion de caracteristicas que se utilizarán en el modelo, la mejor opción es comprender los datos, y comprender las reglas del negocio**

Selecionamos las siguientes variables predictoras (columnas)

- **funded_amnt**             ( monto financiado)  La cantidad total comprometida con ese préstamo en ese momento.
- **loan_term_year**          ( Plazo del credito )
- **int_rate**                ( taza de interes )
- **grade_code**              ( codigo de la calificacion del crédito )
- **purpose_code**            ( codigo del proposito)
- **addr_state_code**         ( codigo de la ciudad)
- **home_ownership_code**     ( codigo situacion casa)
- **annual_inc**              ( ingresos anuales)
- **dti**                     ( Una relación calculada utilizando los pagos de deuda mensuales totales del prestatario )
- **revol_util**              ( cantidad de crédito que el prestatario está utilizando en relación con todo el crédito  )
- **pub_rec_bankruptcies**    ( Número de quiebras de registros públicos )

**Variable dependiente (variable a predecir)**
* **la etiqueta `repaid` será la elegida para el modelo de clasificación**.

**Otros aspectos a considerar**

- Solo las **características que están disponibles antes de que se inicie el préstamo pueden usarse en la clasificación**.
- Las características como **recoveries** (recuperaciones o cargo posterior a la recuperación bruta), **total_rec_prncp** (Principal recibido a la fecha), que solo están disponibles después de que se cierra el préstamo, no deben incluirse en las funciones de entrenamiento.
- Si la clasificación logra una tasa de precisión cercana al 100%, es probable que incluya características que solo estarán disponibles después de que se cierre el préstamo.



In [3]:
X = prestamos_df[['funded_amnt', "int_rate", "grade_code", 'purpose_code', 'addr_state_code',
                 'home_ownership_code', 'annual_inc', 'dti', 'revol_util',
                 'pub_rec_bankruptcies']]

# Variable objetivo o variable a predecir
y = prestamos_df["repaid"]

### **Dividir dataset en conjunto de entrenamiento y prueba**

In [4]:
# Dividimos el dataFrame
# stratify es para que mantenga la misma proporción de clases en ambos conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.4, stratify=y )

In [5]:
# verificamos la cantidad de registros asignados al dataframe de entrenamiento
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11944, 10), (7964, 10), (11944,), (7964,))

## **Escalamiento de variables**

**Por qué es necesario escalar en SVM?**

* SVM usa distancias (en el espacio de características) para construir el hiperplano óptimo.
* Si las variables tienen rangos muy diferentes (por ejemplo, ingresos en miles y tasa de interés en decimales), las de mayor escala dominarán el cálculo.
* Esto afecta:
  * La posición del hiperplano.
  * El margen.
  * El rendimiento del kernel (especialmente RBF, polinomial).

In [6]:
# --- Normalización de variables ---
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## **Definir el modelo SVM y entrenarlo**

In [7]:
# --- Definimos el modelo SVM ---
svm_model = SVC(
    kernel='rbf',          # radial basis function (no lineal)
    C=1.0,                 # penalización por errores
    gamma='scale',         # ajuste automático de gamma
    class_weight='balanced',  # útil si hay desbalance
    probability=True,      # permite obtener probabilidades
    random_state=42
)

# --- Entrenamiento ---
svm_model.fit(X_train_scaled, y_train)

SVC(class_weight='balanced', probability=True, random_state=42)

## **Fase de prueba y Reporte de clasificación**

In [8]:
# --- Predicción ---
y_pred_svm = svm_model.predict(X_test_scaled)

print("Reporte de Clasificación (SVM):")
print(classification_report(y_test, y_pred_svm, target_names=["No Pagado", "Pagado"]))

print("Matriz de Confusión (SVM):")
print(confusion_matrix(y_test, y_pred_svm))

Reporte de Clasificación (SVM):
              precision    recall  f1-score   support

   No Pagado       0.22      0.64      0.33      1177
      Pagado       0.91      0.62      0.73      6787

    accuracy                           0.62      7964
   macro avg       0.57      0.63      0.53      7964
weighted avg       0.81      0.62      0.68      7964

Matriz de Confusión (SVM):
[[ 753  424]
 [2602 4185]]


### **Interpretación**

| Métrica             | Clase No Pagado (0) | Clase Pagado (1) | Explicación                                                                                                                                                |
| ------------------- | ------------------- | ---------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Precision**       | 0.22                | 0.91             | Solo 22 % de las predicciones de *No Pagado* son correctas. Es decir, el modelo genera bastantes falsos positivos (predice “no pagará” aunque sí lo hará). |
| **Recall**          | **0.64**         | 0.62             | Detecta el **64 % de los impagos reales (753)**, una mejora significativa frente a los modelos iniciales.                                                        |
| **F1-score**        | 0.33                | 0.73             | El F1 de la clase minoritaria sigue siendo bajo (por la baja precisión), pero refleja un buen equilibrio entre detección y exactitud.                      |
| **Accuracy global** | 0.62                | —                | El modelo acierta el 62 % de las veces. No es alto, pero el objetivo aquí no es la exactitud global sino **capturar morosos reales**.                      |


## **Análisis AUC-ROC (Área Bajo la Curva- Característica Operativa del Receptor)**

* En machine learning, el término "área bajo la curva" (en inglés, AUC: Area Under the Curve) se refiere comúnmente al área bajo la curva ROC (Receiver Operating Characteristic) (Característica Operativa del Receptor).
* Es una métrica muy utilizada para evaluar el rendimiento de modelos de clasificación binaria.  

**¿Qué es la curva ROC?**  
La curva ROC es un gráfico que muestra la relación entre:
* **Tasa de verdaderos positivos (TPR) o sensibilidad**: proporción de positivos correctamente identificados.
* **Tasa de falsos positivos (FPR)**: proporción de negativos incorrectamente clasificados como positivos.
* Cada punto en la curva representa un umbral diferente de decisión del modelo.

**¿Qué significa el AUC?**
El AUC mide el área total bajo la curva ROC. Su valor está entre 0 y 1:  

* **AUC = 1.0**: Clasificación perfecta.
* **AUC = 0.5**: Rendimiento aleatorio (como lanzar una moneda).
* **AUC < 0.5**: Peor que aleatorio (el modelo se equivoca más de lo que acierta).  



### **Análisis AUC-ROC de Probabilidades de la clase positiva**

In [9]:
##  Probabilidades de la clase positiva ##

from sklearn.metrics import roc_curve, roc_auc_score
import plotly.graph_objects as go

# --- Probabilidades de la clase positiva ("Pagado" = 1) ---
y_prob_svm = svm_model.predict_proba(X_test_scaled)[:, 1]

# --- Calcular AUC-ROC ---
auc = roc_auc_score(y_test, y_prob_svm)
print(f"\n AUC-ROC: {auc:.3f}")

# --- Curva ROC ---
fpr, tpr, thresholds = roc_curve(y_test, y_prob_svm)

# --- Gráfica interactiva con Plotly ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='SVM', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Aleatorio', line=dict(dash='dash')))
fig.update_layout(
    title=f'Probabilidades de la clase positiva - Curva ROC (AUC = {auc:.3f})',
    xaxis_title='Tasa de Falsos Positivos (1 - Especificidad)',
    yaxis_title='Tasa de Verdaderos Positivos (Sensibilidad)',
    width=700, height=500
)
fig.show()



 AUC-ROC: 0.670


### **Interpretación de  la gráfica**

**Curva ROC — Clase positiva (Pagado)**
* Positivo = Pagado (clase 1)
* Negativo = No Pagado (clase 0)
* En este caso:
  * Eje X (Tasa de falsos positivos) = Proporción de No Pagados clasificados erróneamente como Pagados.
  * Eje Y (Tasa de verdaderos positivos) = Proporción de Pagados correctamente detectados.

***Sirve para evaluar qué tan bien el modelo detecta a los que sí pagaron.***  


**Qué muestra la gráfica (ejes y curva)**
* **Eje X**: Tasa de Falsos Positivos (FPR) = FP / (FP + TN). Mide cuántos pagos reales estás marcando como impagos.
* **Eje Y**: Tasa de Verdaderos Positivos (TPR) / Sensibilidad = TP / (TP + FN). Mide cuántos impagos reales detectas.
* **La línea discontinua diagonal (roja)** = comportamiento aleatorio (AUC = 0.5).
* **La línea azul** = rendimiento del modelo SVM: cuanto más se aleja hacia la esquina superior izquierda, mejor discrimina entre clases.

**Interpretación visual**: la curva azul está claramente por encima de la diagonal, por lo tanto, el modelo tiene capacidad para distinguir entre pagadores e impagadores (no es aleatorio).  

**¿Qué nos dice sobre rendimiento (AUC)?**

* El AUC es el área bajo esa curva.
* Si AUC ≈ 0.5 → sin poder discriminatorio.
* AUC entre 0.7 y 0.8 → buen comportamiento.

* Por la forma de la curva (subida razonable y área notable), la AUC está en rango útil (aprox. 0.7x), lo que confirma que el SVM aprende señales relevantes.  

**Nota**: la ROC y AUC miden discriminación global (para todos los umbrales), no la precisión real en un umbral elegido.

**¿qué clase consideró como "positiva"?**
* En el código se calculó y_prob = predict_proba(... )[:, 1] (probabilidad de la clase 1 = Pagado).
Eso significa:
* **La ROC que ves mide la capacidad para detectar la clase “Pagado”** (TPR = recall de Pagado).  

**Si el objetivo es detectar impagos (clase 0), se debe:**
* construir la ROC con la probabilidad de la clase 0 (predict_proba(...)[:,0]) o
* Interpretar la curva actual invertida (FPR/TPR corresponden a Pagado).  

En problemas con clases desbalanceadas donde  interesa más la clase minoritaria (impagos), es mejor evaluar la curva Precision-Recall para la clase 0 o la ROC calculada con la clase 0 como positiva.




###  **Análisis AUC-ROC de Probabilidades de la clase negativa**

In [10]:
# --- Probabilidades de la clase negativa ("Pagado" = 0) ---
y_prob_svm = svm_model.predict_proba(X_test_scaled)[:, 0]

# --- Calcular AUC-ROC ---
auc = roc_auc_score(y_test, y_prob_svm)
print(f"\nAUC-ROC: {auc:.3f}")

# --- Curva ROC ---
fpr, tpr, thresholds = roc_curve(y_test, y_prob_svm)

# --- Gráfica interactiva con Plotly ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='SVM', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Aleatorio', line=dict(dash='dash')))
fig.update_layout(
    title=f'Probabilidades de la clase negativa - Curva ROC (AUC = {auc:.3f})',
    xaxis_title='Tasa de Falsos Positivos (1 - Especificidad)',
    yaxis_title='Tasa de Verdaderos Positivos (Sensibilidad)',
    width=700, height=500
)
fig.show()


AUC-ROC: 0.330


### **Interpretación**

**Curva ROC — Clase negativa (No Pagado)**
* Positivo = No Pagado (clase 0)
* Negativo = Pagado (clase 1)

**Ahora, se invierte la interpretación:**
* Eje X (Tasa de falsos positivos) = Proporción de Pagados clasificados erróneamente como No Pagados.
* Eje Y (Tasa de verdaderos positivos) = Proporción de No Pagados correctamente detectados.
* El modelo (línea azul) está por debajo de la línea aleatoria.
* **AUC = 0.33,** que es menor a 0.5, indica que el modelo está invirtiendo la lógica:
  * predice “Pagado” cuando en realidad era “No Pagado”, y viceversa.

In [12]:
# ===============================================
# 📈 Curvas ROC para ambas clases (positiva y negativa)
# ===============================================

from sklearn.metrics import roc_curve, roc_auc_score
import plotly.graph_objects as go

# --- Probabilidades predichas por el modelo ---
# Asegúrate de tener esto antes:
# svm_model.fit(X_train_scaled, y_train)
# y_prob_svm = svm_model.predict_proba(X_test_scaled)

# Calcular probabilidades y AUC para ambas clases
y_prob = svm_model.predict_proba(X_test_scaled)

# Clase positiva (Pagado = 1)
fpr_pos, tpr_pos, _ = roc_curve(y_test, y_prob[:, 1], pos_label=1)
auc_pos = roc_auc_score(y_test, y_prob[:, 1])

# Clase negativa (No Pagado = 0)
fpr_neg, tpr_neg, _ = roc_curve(y_test, y_prob[:, 0], pos_label=0)
auc_neg = roc_auc_score(y_test, y_prob[:, 0])

# --- Graficar con Plotly ---
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=fpr_pos, y=tpr_pos,
    mode='lines',
    name=f'Clase positiva (Pagado) - AUC = {auc_pos:.3f}',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=fpr_neg, y=tpr_neg,
    mode='lines',
    name=f'Clase negativa (No Pagado) - AUC = {auc_neg:.3f}',
    line=dict(color='green', width=2)
))

# Línea diagonal (modelo aleatorio)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Aleatorio',
    line=dict(color='red', width=1, dash='dash')
))

# --- Personalización del gráfico ---
fig.update_layout(
    title="Comparación de Curvas ROC - Clases Positiva y Negativa",
    xaxis_title="Tasa de Falsos Positivos (1 - Especificidad)",
    yaxis_title="Tasa de Verdaderos Positivos (Sensibilidad)",
    width=900, height=600,
    legend=dict(x=0.6, y=0.05)
)

fig.show()


### **Interpretacion**

* Anteriormente, c**uando se grafico la curva de la clase negativa** (Probabilidades de la clase negativa - Curva ROC), esta se **refleja respecto a la diagonal**, porque estás midiendo sensibilidad y especificidad desde el punto de vista inverso.

* En este nuevo grafico, el área bajo la curva negativa (0.33) no significa que esté “arriba” visualmente —solo que se está midiendo con respecto a una orientación distinta.
* Si se rotara esa curva respecto a la diagonal, se obtendría el mismo comportamiento que esperabas: una por arriba y otra por debajo.  

***Dicho de otra forma:***  
* La clase positiva (Pagado) tiene AUC = 0.67 (mejor que el azar).
* La clase negativa (No Pagado) tiene AUC = 0.33 (peor que el azar, o inversa de la otra).

***Si se invierten los ejes o cambias qué clase consideras “positiva”, las curvas se invierten visualmente.***



## **Optimización de SVM enfocada en detectar creditos NO pagados**  

* **Usar SVC() con class_weight='balanced'** para compensar clases desiguales.
* **Optimizar el F2-score**, que penaliza más los falsos negativos (casos donde el modelo no detecta un impago).
* Evalúar **diferentes valores de C**, kernel, gamma y degree.
* Mostrar el mejor conjunto de hiperparámetros y el desempeño final.

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score, classification_report, confusion_matrix
import numpy as np

# Definimos el F2-score como métrica personalizada (recall más importante que precisión)
f2_scorer = make_scorer(fbeta_score, beta=2, pos_label=0)

# Modelo base
svm = SVC(probability=False, class_weight='balanced', random_state=42)

# Definimos los hiperparámetros a explorar
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3],  # solo afecta al kernel 'poly'
}

# Configuración del GridSearchCV
grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    scoring=f2_scorer,  # optimizamos el F2-score
    cv=5,               # validación cruzada de 5 particiones
    verbose=2,
    n_jobs=-1           # usa todos los núcleos disponibles
)

# Entrenamiento
print("Entrenando GridSearchCV para SVM...\n")
grid_search.fit(X_train_scaled, y_train)

# Resultados
print("\n Búsqueda finalizada.")
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

print("\nMejor puntaje promedio (F2-score):")
print(round(grid_search.best_score_, 4))

# Evaluación con los mejores parámetros
svm_optimo = grid_search.best_estimator_
y_pred_opt = svm_optimo.predict(X_test_scaled)

print("\n Reporte de Clasificación (SVM optimizado para impagos):\n")
print(classification_report(y_test, y_pred_opt, target_names=["No Pagado", "Pagado"]))

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_opt))


Entrenando GridSearchCV para SVM...

Fitting 5 folds for each of 36 candidates, totalling 180 fits

 Búsqueda finalizada.
Mejores hiperparámetros encontrados:
{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}

Mejor puntaje promedio (F2-score):
0.4698

 Reporte de Clasificación (SVM optimizado para impagos):

              precision    recall  f1-score   support

   No Pagado       0.22      0.67      0.33      1177
      Pagado       0.91      0.58      0.71      6787

    accuracy                           0.59      7964
   macro avg       0.56      0.62      0.52      7964
weighted avg       0.81      0.59      0.65      7964

Matriz de Confusión:
[[ 784  393]
 [2847 3940]]


### **Interpretación**

| Métrica                   | Clase | Valor    | Interpretación                                                                                                                                                           |
| ------------------------- | ----- | -------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Recall (No Pagado)**    | 0     | **0.67** | El modelo detecta correctamente **67 % de los impagos reales**, una mejora notable frente a modelos anteriores.                                                          |
| **Precision (No Pagado)** | 0     | 0.22     | De los casos que el modelo predice como impago, solo el 22 % realmente lo son — es decir, hay falsos positivos, pero esto es aceptable en análisis de riesgo crediticio. |
| **Recall (Pagado)**       | 1     | **0.58**    | Baja un poco, lo que indica más confusiones en la clase mayoritaria, pero esto es esperado por el enfoque en la minoritaria.                                             |
| **Accuracy global**       | —     | 0.59     | No es el mejor valor absoluto, pero el objetivo aquí era **recuperar más impagos**, no maximizar el accuracy.                                                            |
| **F2-score global**       | —     | ≈ 0.47   | Confirma que el modelo está optimizado para recall.                                                                                                                      |  


